In [1]:
import torch    
torch.manual_seed(123)
import random
random.seed(123)

import torch.nn as nn
import os
import shutil
import itertools

from chofer_torchex.utils.data.collate import dict_sample_target_iter_concat
from chofer_torchex.utils.functional import collection_cascade, cuda_cascade
from chofer_tda_datasets import Reddit_12K
from jmlr_2018_code.utils import *
from sklearn.model_selection import ShuffleSplit
from chofer_torchex.nn.slayer import SLayerRationalHat, LinearRationalStretchedBirthLifeTimeCoordinateTransform, prepare_batch


from collections import Counter, defaultdict

%matplotlib notebook



from torch.utils.data import DataLoader

os.environ['CUDA_VISIBLE_DEVICES'] = str(1)

def save_extract_birth_times(tensor):
    if tensor.ndimension() == 0:
        return tensor
    else:
        return tensor[:, 0].unsqueeze(dim=1)

class RedditCollate:   
    def __init__(self, dataset, cuda=True):
        self.cuda = cuda
        self.dataset = dataset
        
    def __call__(self, sample_target_iter):
        x, y = dict_sample_target_iter_concat(sample_target_iter)

        for k in self.dataset.keys_not_essential:
            batch_view = x[k]
            x[k] = prepare_batch(batch_view, 2)
            
        for k in self.dataset.keys_essential:
            batch_view = x[k]
            x[k] = prepare_batch(batch_view, 1)            

        y = torch.LongTensor(y)    

        if self.cuda:
            # Shifting the necessary parts of the prepared batch to the cuda
            x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                 for k, v in x.items()}

            y = y.cuda()

        return x, y
    

class train_env:
    n_epochs = 100
    lr_initial = 0.01
    lr_epoch_step = 10
    batch_size = 100
    train_size = 0.9
    nu = 0.01
    momentum = 0.5


In [2]:
dataset = Reddit_12K(root_dir='/scratch1/chofer/jmlr2018_data/')
dataset.keys_essential = ('DegreeVertexFiltration_dim_0_essential', 'DegreeVertexFiltration_dim_1_essential')
dataset.keys_not_essential = ('DegreeVertexFiltration_dim_0',)
dataset.keys_of_interrest = dataset.keys_essential + dataset.keys_not_essential

Found data!


In [3]:
def reduce_essential_dgms(x):
    for k in dataset.keys_essential:
        x[k] = save_extract_birth_times(x[k])
        
    return x

def coordinate_transform(x):
    t = LinearRationalStretchedBirthLifeTimeCoordinateTransform(nu=train_env.nu)
    for k in dataset.keys_not_essential:
        x[k] = t(x[k])
        
    return x

dataset.data_transforms = \
[
    lambda x: {k: x[k] for k in dataset.keys_of_interrest},
    numpy_to_torch_cascade,
    reduce_essential_dgms,
    coordinate_transform    
]

In [4]:
reddit_collate = RedditCollate(dataset)         

In [ ]:
def LinearCell(n_in, n_out):
    m = nn.Sequential(nn.Linear(n_in, n_out), 
                      nn.BatchNorm1d(n_out), 
                      nn.ReLU())
    m.out_features = m[0].out_features
    return m


def Slayer(n_elements, point_dim):
    return SLayerRationalHat(n_elements, point_dimension=2, radius_init=0.25)   


class Reddit12KModel(nn.Module):    
    def __init__(self):
        super().__init__()  
        
        dim_0_n_elements = 400
        dim_0_ess_n_elements = 100
        dim_1_ess_n_elements = 100
        
        self.dim_0 = Slayer(dim_0_n_elements, 2)
        self.dim_0_ess = Slayer(dim_0_ess_n_elements, 1)
        self.dim_1_ess = Slayer(dim_1_ess_n_elements, 1)      
        
        self.dim_0_linear =     LinearCell(dim_0_n_elements, int(dim_0_n_elements/2))
        self.dim_0_ess_linear = LinearCell(dim_0_ess_n_elements, int(dim_0_ess_n_elements/2))
        self.dim_1_ess_linear = LinearCell(dim_1_ess_n_elements, int(dim_1_ess_n_elements/2))
        
        cls_in_size = self.dim_0_linear.out_features + self.dim_0_ess_linear.out_features + self.dim_1_ess_linear.out_features
        n_1 = int(cls_in_size/2)
#         cls_in_size = self.dim_0_linear.out_features + self.dim_0_ess_linear.out_features + self.dim_1_ess_linear.out_features
        self.cls = nn.Sequential(                               
#                                nn.BatchNorm1d(cls_in_size),
#                                nn.ReLU(),
#                                nn.Dropout(0.1),
                               LinearCell(cls_in_size, n_1),
#                                nn.Dropout(0.1),
                               LinearCell(n_1, int(n_1/2)),
#                                nn.Dropout(0.1),
                               LinearCell(int(n_1/2), int(n_1/4)),
                               nn.Linear(int(n_1/4), 11))
                         
    def forward(self, x):
        x_dim_0 = self.dim_0(x['DegreeVertexFiltration_dim_0'])        
        x_dim_0 = self.dim_0_linear(x_dim_0)
        
        x_dim_0_ess = self.dim_0_ess(x['DegreeVertexFiltration_dim_0_essential'])   
        x_dim_0_ess = self.dim_0_ess_linear(x_dim_0_ess)
        
        x_dim_1_ess = self.dim_1_ess(x['DegreeVertexFiltration_dim_1_essential'])       
        x_dim_1_ess = self.dim_1_ess_linear(x_dim_1_ess)
        
        x = torch.cat([x_dim_0, x_dim_0_ess, x_dim_1_ess], dim=1)
        x = self.cls(x)
            
        return x
        
    def centers_init(self):
        dim_0 = []
        while len(dim_0) != self.dim_0.centers.size(0):
            x = random.uniform(0, 1)
            y = random.uniform(0, 1)
            if y + x < 1:
                dim_0.append((x,y))
#         for i in range(self.dim_0.centers.size(0)):
#             x = random.uniform(0, 1)
#             y = random.uniform(0, 1-x)
#             dim_0.append((x,y))
        self.dim_0.centers.data = torch.Tensor(dim_0)
        
        self.dim_0_ess.centers.data.uniform_(0, 1)
        self.dim_1_ess.centers.data.uniform_(0, 1)
            
            
       

        

In [ ]:
stats_of_runs = []
def experiment():      
    splitter = ShuffleSplit(n_splits=10, 
                            train_size=train_env.train_size, 
                            test_size=1-train_env.train_size, 
                            random_state=123)
    train_test_splits = list(splitter.split(X=dataset.labels, y=dataset.labels))
    train_test_splits = [(train_i.tolist(), test_i.tolist()) for train_i, test_i in train_test_splits]
    
    for run_i, (train_i, test_i) in enumerate(train_test_splits):
        print('')
        print('Run', run_i)
        
        model = Reddit12KModel()
        model.centers_init()
        model.cuda()

        stats = defaultdict(list)
        stats_of_runs.append(stats)      
        
        
        opt=torch.optim.SGD(model.parameters(), lr=train_env.lr_initial, momentum=train_env.momentum)

        for i_epoch in range(1, train_env.n_epochs+1):      

            model.train()

            train_sampler = [i for i in train_i] 
            random.shuffle(train_sampler)    
            
            dl_train = DataLoader(dataset,
                              batch_size=train_env.batch_size, 
                              collate_fn=reddit_collate,
                              sampler=train_sampler)

            dl_test = DataLoader(dataset,
                                 batch_size=train_env.batch_size, 
                                 collate_fn=reddit_collate, 
                                 sampler=test_i)

            epoch_loss = 0

            if i_epoch % train_env.lr_epoch_step == 0:
                adapt_lr(opt, lambda lr: lr*0.5)

            for i_batch, (x, y) in enumerate(dl_train, 1):              

                y = torch.autograd.Variable(y)

                def closure():
                    opt.zero_grad()
                    y_hat = model(x)            
                    loss = nn.functional.cross_entropy(y_hat, y)   
                    loss.backward()
                    return loss

                loss = opt.step(closure)

                epoch_loss += float(loss)
                stats['loss_by_batch'].append(float(loss))
                stats['centers'].append(model.dim_0.centers.data.cpu().numpy())

                print("Epoch {}/{}, Batch {}/{}".format(i_epoch, train_env.n_epochs, i_batch, len(dl_train)), end="       \r")

            stats['train_loss_by_epoch'].append(epoch_loss/len(dl_train))

            print("\n\r testing...")
            model.eval()    
            true_samples = 0
            seen_samples = 0
            epoch_test_loss = 0
            for i_batch, (x, y) in enumerate(dl_test):

                y_hat = model(x)
                epoch_test_loss += float(nn.functional.cross_entropy(y_hat, torch.autograd.Variable(y.cuda())).data)

                y_hat = y_hat.max(dim=1)[1].data.long()

                true_samples += (y_hat == y).sum()
                seen_samples += y.size(0)  

            stats['test_accuracy'].append(true_samples/seen_samples)
            stats['test_loss_by_epoch'].append(epoch_test_loss/len(dl_test))
            print(true_samples/seen_samples)       
        
experiment()        


Run 0
Epoch 1/100, Batch 108/108       
 testing...
0.3126571668063705
Epoch 2/100, Batch 108/108       
 testing...
0.38474434199497065
Epoch 3/100, Batch 108/108       
 testing...
0.38809723386420786
Epoch 4/100, Batch 108/108       
 testing...
0.32858340318524726
Epoch 5/100, Batch 108/108       
 testing...
0.38474434199497065
Epoch 6/100, Batch 108/108       
 testing...
0.38390611902766136
Epoch 7/100, Batch 108/108       
 testing...
0.40150880134115674
Epoch 8/100, Batch 108/108       
 testing...
0.41743503772003354
Epoch 9/100, Batch 108/108       
 testing...
0.41827326068734283
Epoch 10/100, Batch 108/108       
 testing...
0.42497904442581724
Epoch 11/100, Batch 108/108       
 testing...
0.42749371332774516
Epoch 12/100, Batch 108/108       
 testing...
0.41827326068734283
Epoch 13/100, Batch 108/108       
 testing...
0.4258172673931266
Epoch 14/100, Batch 108/108       
 testing...
0.4157585917854149
Epoch 15/100, Batch 108/108       
 testing...
0.4107292539815591
E

0.4308466051969824
Epoch 46/100, Batch 108/108       
 testing...
0.4300083822296731
Epoch 47/100, Batch 108/108             
 testing...
0.4266554903604359
Epoch 48/100, Batch 108/108       
 testing...
0.41827326068734283
Epoch 49/100, Batch 108/108       
 testing...
0.4191114836546521
Epoch 50/100, Batch 108/108       
 testing...
0.4300083822296731
Epoch 51/100, Batch 108/108       
 testing...
0.43671416596814755
Epoch 52/100, Batch 108/108       
 testing...
0.42749371332774516
Epoch 53/100, Batch 108/108       
 testing...
0.42330259849119867
Epoch 54/100, Batch 108/108       
 testing...
0.4191114836546521
Epoch 55/100, Batch 108/108       
 testing...
0.42414082145850796
Epoch 56/100, Batch 108/108             
 testing...
0.42749371332774516
Epoch 57/100, Batch 108/108       
 testing...
0.42078792958927075
Epoch 58/100, Batch 108/108       
 testing...
0.42497904442581724
Epoch 59/100, Batch 108/108       
 testing...
0.4333612740989103
Epoch 60/100, Batch 108/108       
 t

In [10]:
print(np.mean([np.mean(s['test_accuracy'][-10:]) for s in stats_of_runs]))
[np.mean(s['test_accuracy'][-10:]) for s in stats_of_runs]

0.430695725063


[0.41584241408214578,
 0.46236378876781214,
 0.42506286672254817,
 0.41106454316848284,
 0.43512154233025979,
 0.41299245599329415,
 0.43285834031852471,
 0.44283319362950546,
 0.44409052808046939,
 0.42472757753562451]

In [ ]:
plt.figure()
stats = stats_of_runs[0]
c_start = stats['centers'][0]
c_end = stats['centers'][-1]

plt.plot(c_start[:,0], c_start[:, 1], 'bo', label='center initialization')
plt.plot(c_end[:,0], c_end[:, 1], 'ro', label='center learned')

all_centers = numpy.stack(stats['centers'], axis=0)
for i in range(all_centers.shape[1]):
    points = all_centers[:,i, :]
    plt.plot(points[:, 0], points[:, 1], '-k')
    
plt.legend()

plt.figure()
plt.plot(stats['train_loss_by_epoch'], label='train_loss')
plt.plot(stats['test_loss_by_epoch'], label='test_loss')
plt.plot(stats['test_accuracy'], label='test_acccuracy')

plt.legend()
plt.show()

In [ ]:
print(model.dim_0.exponent)
print(model.dim_0.sharpness)